### Data Analysis - Interface & Preparation Model
> This system provides pre-processed access to the Active Campaign Member Data with the Addition of Further Analytics and Fields generated by the system through Pandas/Numpy Python

> The Data Pre-Processing and Data Model Preparation Stage must be Run and Executed prior to running any Scripts / Queries

> Individual Scripts Can Be Run To Visualise Data or To Download Data Respectively

### Data Pre-Processing and Data Model Preparation

In [67]:
import requests
import json
import csv
import pandas

import re
import time
import math
import numpy

from datetime import datetime

# Constants
number_of_contacts = 9000
records_per_query = 100
list_of_site_names = ["Wimbletech","TheWorkary - Chiswick","Rivertech","TheWorkary - Brompton","TheWorkary - NottingHill","TheWorkary - Hanwell","TheWorkary - Chelsea","TheWorkary - Welling","TheWorkary - Caterham","TheWorkary - Maidenhead", "TheWorkary - Barnet Pop Up","TheWorkary - Avonmore","TheWorkary - Rainham"]
list_of_site_postcodes = ["SW197NB","W42AB","WD31HP","SW50BS","W24EW","W71PD","SW35EZ","DA163PA","CR36TR","SL61JX",None,"W148TG","RM139YJ"]
postcode_table = pandas.DataFrame({'SiteName':list_of_site_names,'SitePostcode':list_of_site_postcodes})

postcode_regex_filter = "^(([A-Z]|[a-z]){1,2}[0-9]{1,2}([A-Z]|[a-z]){0,1}\s{0,1}[0-9]{1}([A-Z]|[a-z]){2})$"
compiled_postcode_regex = re.compile(postcode_regex_filter)

# Calculated Haversine Straight Line Distance in Kilometers
def calculate_km_distance_between_points(lat1,lon1,lat2,lon2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon/2)**2
    c = 2 * numpy.arcsin(numpy.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    km_distance = r*c
    
    return numpy.round(km_distance)

# Calculated Haversine Straight Line Distance in Miles
def calculate_miles_distance_between_points(lat1,lon1,lat2,lon2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(numpy.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon/2)**2
    c = 2 * numpy.arcsin(numpy.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    km_distance = r*c
    
    #6371*(2 * math.asin(math.sqrt(math.sin((math.radians(lat2)-math.radians(lat1))/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin((math.radians(lon2)-math.radians(lon1))/2)**2)))

    return numpy.round(km_distance*0.621371)

def print_only_completed_results():
  df = pandas.read_csv('Active Campaign - Contacts Report.csv',dtype='unicode')
  print(len(df))
  print(len(df.columns))

  # Select Only Desired Columns
  df2_only_desired_columns = df.iloc[:,[0,1,2,3,12,17,19,25,52,54,60,79,100]]
  # Drop Any Columns with Any Missing Rows
  return df2_only_desired_columns.dropna()

def print_only_complete_records():
  df = pandas.read_csv('Active Campaign - Contacts Report.csv')
  print(len(df))
  print(len(df.columns))

  # Select Only Desired Columns
  df2_only_desired_columns = df.iloc[:,[0,1,2,3,12,17,19,25,52,54,60,79,100]]
  print(df2_only_desired_columns.columns)
  # Drop Any Columns with Any Missing Rows
  return df2_only_desired_columns.dropna()

def produce_data_frame():
  df = pandas.read_csv('Active Campaign - Contacts Report.csv')
  print(len(df))
  print(len(df.columns))

  # # Tapi.postcodes.io/postcodes/
  #   url = https://wimbletech.api-us1.com/api/3/contacts?api_key=03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0&include=fieldValues
  # field_values_url = "https://api.postcodes.io/postcodes/"
  # headers = {"Accept": "application/json","Api-Token":"03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0"}
  # querystring = {"status":"-1","limit": records_per_query,"offset": records_per_query*counter}
  # response = requests.request("GET", field_values_url, headers=headers, params=querystring)
  # contacts_output = json.loads(response.text)["contacts"]
  
    
  # Select Only Desired Columns from the Database
  df2_only_desired_columns = df.iloc[:,[0,1,2,3,12,17,19,25,52,54,60,79,100]]
  
  # Create Value Counts For Each Site
  df2_site_counts = df2_only_desired_columns['*Venue - Site (final)'].value_counts().to_frame()
  print(df2_site_counts.columns)

  # Merge Aggregated Count Table with Original Table
  df3_merged_table = df2_only_desired_columns.merge(df2_site_counts,how='outer',left_on='*Venue - Site (final)',right_index=True)
  
  # Merge Post-Codes On To Table
  df4_postcode_included_table = df3_merged_table.merge(postcode_table,how='outer',left_on='*Venue - Site (final)',right_on="SiteName")

  # Log 

  # Insert Flag Columns for Null Values
  df4_postcode_included_table['LastDateIncluded'] = df4_postcode_included_table['*Last Date'].notnull()
  df4_postcode_included_table['StartDateIncluded'] = df4_postcode_included_table['*Start Date'].notnull()
  
  txt = "HP52DA"
  x = re.search("^([A-Z]{1,2}[0-9]{1,2}[A-Z]{0,1}\s{0,1}[0-9]{1}[A-Z]{2})$", txt)

  if x:
      print("YES! We have a match!")
  else:
      print("No match")
        
  print("Regex = " + str(re.search("^([A-Z]{1,2}[0-9]{1,2}[A-Z]{0,1}\s{0,1}[0-9]{1}[A-Z]{2})$", "WD25 0JL")))
  df4_postcode_included_table['StringConvertedRegex'] = df4_postcode_included_table['*Postcode']
  
  df_long_lat_points = pandas.read_csv('Postcode - Latitude With No Nulls.csv',dtype='unicode')
     

  df4_postcode_included_table['VenueSiteFinalIncluded'] = df4_postcode_included_table['*Venue - Site (final)'].notnull()
  df4_postcode_included_table['PostcodeIncluded'] = df4_postcode_included_table['*Postcode'].notnull()

  # Could Add In Prospect / Member Live Field If Possible
  # Write Search Queries That Return Filtered Fields for Dates Within Different Time Regions < 30 days >

  # Output the file
  df4_postcode_included_table.filter(items=["SitePostcode","*Postcode"])
    
  long_lat_points = pandas.read_csv('Postcode - Latitude With No Nulls.csv',dtype='unicode')[["Postcode","Latitude","Longitude"]]
  site_long_lat_data = pandas.read_csv('Postcode - OUTPUT - Site Latitude and Longtitude.csv',dtype='unicode')[["Postcode","Latitude","Longitude"]]
    

  df_5_member_longlat_included = df4_postcode_included_table.merge(long_lat_points,how='left',left_on='*Postcode',right_on="Postcode")
  df_6 = df_5_member_longlat_included.merge(site_long_lat_data,how='left',left_on='SitePostcode',right_on="Postcode")

  # Drop Any Columns with Any Missing Rows
  df_6["km_between_points"] = df_6.apply(lambda x : calculate_km_distance_between_points(float(x["Latitude_x"]),float(x["Longitude_x"]),float(x["Latitude_y"]),float(x["Longitude_y"])),axis=1)
  df_6["miles_between_points"] = df_6.apply(lambda x : calculate_miles_distance_between_points(float(x["Latitude_x"]),float(x["Longitude_x"]),float(x["Latitude_y"]),float(x["Longitude_y"])),axis=1)
  df_6.rename(columns={"Postcode_x": "SitePostcode_x" , "Latitude_x": "SiteLatitude_x","Longitude_x":"SiteLongitude_x","Postcode_y":"HomePostcode_y","Latitude_y":"Home_Latitude_y","Longitude_y":"Home_Longitude_y"})
  
  df_6.to_csv('ZohoOutput' + str(time.time()) + ".csv",header = True,index=False)
  return df_6

def printout_contacts_report(dataframe):
  df = pandas.read_csv('/Active Campaign - Contacts Report.csv',dtype='unicode')
  print(len(df))
  print(len(df.columns))

  # Select Only Desired Columns
  df2_only_desired_columns = df.iloc[:,[0,1,2,3,12,17,19,25,52,54,60,79,100]].sort_values(['*Last Date'])
  # Create Additional Flag Fields Indicating Nan Values
  df2_nan_flags = df2_only_desired_columns.isna()

  return df2_only_desired_columns
  # Conduct A Grouped Count Of How Many Members Are In Samples For Each Column (After Excluding Nulls)
    # Where Start Date Is Before Today and L
  above_35 = dataframe[dataframe["*Start Date"] > 35]

  # Conduct A Grouped Count Of How Many Members Are In Samples For Each Column (Non-Nulls Group By Sites)

  # Conduct A Count Of Start Dates In The Future
    # 1. List Of New Starter Members - Where Start Date Is After Today
    # 2. Count Of New Starter Members By Site - Where Start Date Is After Today Grouped By Site
  # Conduct A Count of Start Dates In The Last 30 Days
    # 3. Count Of New Starter Members By Site - Where the Start Date Is Within the Last 30 Days
    # 4. Count Of New Starter Members By Site - Where the Start Date Is Within The Last 30 Days - Grouped By Site
  # Conduct A Count of Leaving Dates In The Next 30 Days
    # 5. Count Of New Starter Members By Site - Where the Leave Date Is Within The Next 30 Days
    # 6. Count Of New Starter Members By Site - Where the Start Date Is Within The Next 30 Days - Grouped By Site
  # Conduct A Count of Start Dates In The Pass 90 Days
    # 7. Count Of New Starter Members By Site - Where the Start Date Is Within The Next 90 Days
    # 8. Count Of New Starter Members By Site - Where the Start Date Is Within The Next 90 Days - Grouped By Site=
  
  # Historical Data Analytics
  # Membership Duration Time - Length Of Stay

  # Conduct A Count of Postcode 
output = produce_data_frame()


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (11,44,64,73,78,82,83,84,85,86,89,90,92,93,94,95,96,113,115,121,122,124,125,126,150,157,163,191,207,215,216,217,221,234,239,248,249) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


8984
251
Index(['*Venue - Site (final)'], dtype='object')
YES! We have a match!
Regex = <re.Match object; span=(0, 8), match='WD25 0JL'>


### Redundant Code Below is Subject to Clarifications from the Active Campaign Develop



In [ ]:
def retrieve_all_contacts(global_list_object):
  url = "https://wimbletech.api-us1.com/api/3/contacts"
  for counter in range(number_of_contacts/100):
    headers = {"Accept": "application/json","Api-Token":"03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0"}
    querystring = {"status":"-1","limit": records_per_query,"offset": records_per_query*counter}
    response = requests.request("GET", url, headers=headers, params=querystring)
    contacts_output = json.loads(response.text)["contacts"]
    print(querystring)
    global_list_object = global_list_object + contacts_output
    
    #Introduce Rate Limits
    if(counter+1 % 5 == 0):
      print("Pause for Rate-Limit")
      sleep(2)
      console.log("Pause for Rate-Limit")

    # Write To File
  with open('names.csv', 'w', newline='') as csvfile:
    print("Field Names = " + str(contacts_db[0].keys()))
    fieldnames = contacts_output[0].keys()

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for item in contacts_db:
      writer.writerow(item)
      sleep(0.5)

def retrieve_all_field_values(global_list_object):
  for counter in range(number_of_contacts/100):
    field_values_url = "https://wimbletech.api-us1.com/api/3/fieldValues"
    headers = {"Accept": "application/json","Api-Token":"03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0"}
    querystring = {"status":"-1","limit": records_per_query,"offset": records_per_query*counter}
    response = requests.request("GET", field_values_url, headers=headers, params=querystring)
    contacts_output = json.loads(response.text)["contacts"]
    print(querystring)
    global_list_object = global_list_object + contacts_output
      
    #Introduce Rate Limits
    if(counter+1 % 5 == 0):
      print("Pause for Rate-Limit")
      sleep(2)
      console.log("Pause for Rate-Limit")

    # Write To File
    with open('fieldnames.csv', 'w', newline='') as csvfile:
      print("Field Names = " + str(contacts_db[0].keys()))
      fieldnames = contacts_output[0].keys()

      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

      writer.writeheader()

      for item in contacts_db:
        writer.writerow(item)
        sleep(0.5)

def retrieve_field_values(id):
  url = "https://wimbletech.api-us1.com/api/3/fieldValues/" + str(id)
  headers = {"Accept": "application/json"}
  headers = {"Accept": "application/json","Api-Token":"03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0"}
  querystring = {"status":"-1","limit": records_per_query,"offset": records_per_query*counter}
  response = requests.request("GET", url, headers=headers, params=querystring)
  print(response.text)

def retrieve_field_values_2():
  url = "https://wimbletech.api-us1.com/api/3/fields"
  querystring = {"limit":"1"}
  headers = {"Accept": "application/json","Api-Token":"03607701a84655ffeffb618671d5f4be937c574b9bf1fe11ad19b157b1aef835d90768f0"}
  response = requests.request("GET", url, headers=headers, params=querystring)
  print(response.text)

def retrieve_all_lists():
  url = "https://wimbletech.api-us1.com/api/3/lists"
  headers = {"Accept": "application/json"}
  response = requests.request("GET", url, headers=headers)
  print(response.text)

### Utility Functions

In [ ]:
def extract_postcodes_api():
    url = "https://api.postcodes.io/postcodes"
    with open('Postcode - Copy of JSON Paramater Packaging.csv', newline='') as csvfile:
        postcodes = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in postcodes:
            print("Row = " + str(row))
            #response = requests.post(url,data = row)
            #json_output = json.loads(response)
            #print(response.text)
            
def extract_postcodes_file():
    long_lat_points = pandas.read_csv('Postcode - Latitude With No Nulls.csv',dtype='unicode')
    return long_lat_points

### Run Queries and Views on The Prepared Model

In [68]:
output

,*Venue - Site (final),ID,Email,First Name,Last Name,*Title,*Venue - Site (Preference),*D.O.B.,*Postcode,*Venue - Site (final)_x,...,VenueSiteFinalIncluded,PostcodeIncluded,Postcode_x,Latitude_x,Longitude_x,Postcode_y,Latitude_y,Longitude_y,km_between_points,miles_between_points
0,Rivertech,12569.0,NaN,Vikram,Chudasama,Mr,Rivertech (BasingHouse - Rickmansworth),NaN,NaN,Rivertech,...,True,False,NaN,NaN,NaN,WD31HP,51.639328,-0.46847207,NaN,NaN
1,Rivertech,997.0,abbey.robinson@stacktechnology.co.uk,Abbey,Robinson,NaN,NaN,NaN,NaN,Rivertech,...,True,False,NaN,NaN,NaN,WD31HP,51.639328,-0.46847207,NaN,NaN
2,Rivertech,5209.0,abhay.shah@caremark.co.uk,Abhay,Shah,Mr,NaN,NaN,HP52DA,Rivertech,...,True,True,HP52DA,51.721502,-0.61685707,WD31HP,51.639328,-0.46847207,14.0,9.0
3,Rivertech,4418.0,abhayshah@hotmail.co.uk,Abhay,Shah,Mr,Rivertech (BasingHouse - Rickmansworth),NaN,NaN,Rivertech,...,True,False,NaN,NaN,NaN,WD31HP,51.639328,-0.46847207,NaN,NaN
4,Rivertech,4176.0,accounts@ipcomms.eu,Carel,Solomon,Mr,NaN,NaN,NaN,Rivertech,...,True,False,NaN,NaN,NaN,WD31HP,51.639328,-0.46847207,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9252,TheWorkary - Rainham,164.0,hands4healing@uwclub.net,Diane,Tyrrell,Mrs,Interest in new sites - Rainham,2016-01-23,NaN,TheWorkary - Rainham,...,True,False,NaN,NaN,NaN,RM139YJ,51.517575,0.19117886,NaN,NaN
9253,TheWorkary - Rainham,368.0,lucie@opalcontracts.co.uk,Lucie,Reeve,Mrs,Rainham - Fixed Single Desk -,1991-06-26,B7 4TE,TheWorkary - Rainham,...,True,True,B7 4TE,52.492836,-1.8690651,RM139YJ,51.517575,0.19117886,178.0,111.0
9254,TheWorkary - Rainham,228.0,mark@mmadvertising.co.uk,Mark,Jones,Mr,Interested in new sites COMING SOON - see belo...,1985-10-22,NaN,TheWorkary - Rainham,...,True,False,NaN,NaN,NaN,RM139YJ,51.517575,0.19117886,NaN,NaN
9255,TheWorkary - Rainham,259.0,mattmounsey@live.co.uk,Matt,Mounsey,Mr,Rainham - Flexi Desk -,1981-09-22,legal,TheWorkary - Rainham,...,True,True,NaN,NaN,NaN,RM139YJ,51.517575,0.19117886,NaN,NaN
